In [14]:
from langgraph.graph import Graph,MessagesState,StateGraph,END,START
from langchain_core.messages import HumanMessage,SystemMessage
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

from IPython.display import Image,display
from typing import TypedDict,Annotated,Sequence
import operator,json
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

groq_key = os.getenv("groq_api_key")
llm = ChatGroq(model_name ="gemma2-9b-it", api_key=groq_key)


In [5]:
llm.invoke("hi")

AIMessage(content='Hello! 👋  How can I help you today? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26, 'completion_time': 0.027272727, 'prompt_time': 0.001899597, 'queue_time': 0.24918127199999998, 'total_time': 0.029172324}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7725c519-4ca7-4788-af14-1d4c05790082-0', usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26})

In [7]:
@tool
def multiply(first_number: int, second_number : int)->int:
    """ Multiply two integers"""
    return first_number * second_number


In [13]:
# multiply({"first_number":12,"second_number":2})
# multiply.invoke({"first_number":12,"second_number":3})

In [ ]:
# @tool
# def search(query:str):
#     """Perform the web search on the user query"""
#     tavily = TavilySearchResults()
#     result = tavily.invoke(query)
#     return result

In [19]:
@tool
def search(query:str):
    """ Search web based on the user query"""
    dgdgs = DuckDuckGoSearchRun()
    result = dgdgs.invoke(query)
    return result


In [22]:
# search("who is the current president of the USA?")

In [23]:
tools = [search,multiply]

<!-- retrieving function names and arguments for tool name and question -->

In [25]:
model_with_tools = llm.bind_tools(tools)

In [48]:
tool_mapping = {tool.name :tool for tool in tools}


In [49]:
tool_mapping

{'search': StructuredTool(name='search', description='Search web based on the user query', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000022F3084F420>),
 'multiply': StructuredTool(name='multiply', description='Multiply two integers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000022F2E6199E0>)}

In [51]:
response = model_with_tools.invoke("who is Indian current  prime minister?")

In [52]:
tool_details = response.additional_kwargs.get("tool_calls")

In [53]:
tool_details

[{'id': 'call_esea',
  'function': {'arguments': '{"query":"who is the current prime minister of India"}',
   'name': 'search'},
  'type': 'function'}]

In [54]:
tool_details[0]["function"]["name"]

'search'

In [56]:
tool_details[0]["function"]["arguments"]

'{"query":"who is the current prime minister of India"}'

In [26]:
class AgentState(TypedDict):
    message: Annotated[Sequence[BaseMessage],operator.add]

In [27]:
def invoke_model(state:AgentState):
    messages = state["messages"]
    question = messages[-1] ## fetching the usr question
    return {"messages":[model_with_tools.invoke(question)]}

In [28]:
def invoke_tool(state: AgentState):
    pass


In [29]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls",[])
    if len(tool_calls):
        return "tool"
    else:
        return "end"

In [45]:
graph = StateGraph(AgentState)

graph.add_node("ai_assistant",invoke_model)
graph.add_node("tool",invoke_tool)

In [46]:
graph.add_conditional_edges("ai_assistant",router,{"tool":tool,"end":END,})
graph.add_edge("tool",END)
graph.set_entry_point("ai_assistant")

In [47]:
graph.compile()

ValueError: At 'ai_assistant' node, 'router' branch found unknown target '<function tool at 0x0000022F25EF6480>'